In [1]:
import pandas as pd
import time
from datetime import datetime, timedelta
import json
import requests
import re
import numpy as np
import matplotlib as plt 
from bs4 import BeautifulSoup
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
#for outliers
def outlier_detection_iqr(df):
    df_copy = df.copy()
    for col in df_copy.keys():
        if df_copy[col].dtype != object:
            Q1 = np.percentile(df_copy[col],25)
            Q3 = np.percentile(df_copy[col],75)
            IQR = Q3 - Q1
            IQR_range = 1.5 * IQR
            df_copy.loc[(df_copy[col] < Q1-IQR_range) | (df_copy[col] > Q3+IQR_range),[col]] = np.nan
    
    return df_copy

In [4]:
df = pd.read_csv("NFLPlayerDetails.csv")
df

,Player Name,Salary,Experience Years,Team,Position,Age,Drafted Details,College,Agent,Player Status
0,Dak Prescott,"$450,000",5 Years,Cowboys,QB,27-24d,"Round 4 (#135 overall), 2016",Mississippi State,Todd France (CAA Sports),Active
1,Aaron Rodgers,"$230,000",16 Years,Packers,QB,37-20d,"Round 1 (#24 overall), 2005",California,David Dunn (Athletes First),Active
2,Matthew Stafford,"$3,100,000",12 Years,Rams,QB,33-15d,"Round 1 (#1 overall), 2009",Georgia,Tom Condon (CAA Sports),Active
3,Joey Bosa,"$450,000",5 Years,Chargers,DE,25-11d,"Round 1 (#3 overall), 2016",Ohio State,Brian Ayrault (CAA Sports),Active
4,Jimmy Garoppolo,"$420,000",7 Years,49ers,QB,29-20d,"Round 2 (#62 overall), 2014",Eastern Illinois,Don Yee (Yee & Dubin Sports),Active
...,...,...,...,...,...,...,...,...,...,...
4267,Colby Gossett,-,2 Years,Browns,G,25-29d,"Round 6 (#213 overall), 2018",Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active
4268,Jeremiah Valoaga,"$300,883",3 Years,Raiders,DE,26-7d,NaN,"2017, Lions",Nevada-Las Vegas,Inactive
4269,Hunter Sharp,"$23,750",1 Years,Bengals,WR,27-27d,NaN,"2016, Eagles",Utah State,Inactive
4270,LaTroy Lewis,"$86,400",1 Years,Titans,ILB,27-13d,NaN,"2017,",Tennessee,Inactive


In [5]:
df["Drafted Details"].fillna("Undrafted, Undrafted", inplace = True)
df

,Player Name,Salary,Experience Years,Team,Position,Age,Drafted Details,College,Agent,Player Status
0,Dak Prescott,"$450,000",5 Years,Cowboys,QB,27-24d,"Round 4 (#135 overall), 2016",Mississippi State,Todd France (CAA Sports),Active
1,Aaron Rodgers,"$230,000",16 Years,Packers,QB,37-20d,"Round 1 (#24 overall), 2005",California,David Dunn (Athletes First),Active
2,Matthew Stafford,"$3,100,000",12 Years,Rams,QB,33-15d,"Round 1 (#1 overall), 2009",Georgia,Tom Condon (CAA Sports),Active
3,Joey Bosa,"$450,000",5 Years,Chargers,DE,25-11d,"Round 1 (#3 overall), 2016",Ohio State,Brian Ayrault (CAA Sports),Active
4,Jimmy Garoppolo,"$420,000",7 Years,49ers,QB,29-20d,"Round 2 (#62 overall), 2014",Eastern Illinois,Don Yee (Yee & Dubin Sports),Active
...,...,...,...,...,...,...,...,...,...,...
4267,Colby Gossett,-,2 Years,Browns,G,25-29d,"Round 6 (#213 overall), 2018",Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active
4268,Jeremiah Valoaga,"$300,883",3 Years,Raiders,DE,26-7d,"Undrafted, Undrafted","2017, Lions",Nevada-Las Vegas,Inactive
4269,Hunter Sharp,"$23,750",1 Years,Bengals,WR,27-27d,"Undrafted, Undrafted","2016, Eagles",Utah State,Inactive
4270,LaTroy Lewis,"$86,400",1 Years,Titans,ILB,27-13d,"Undrafted, Undrafted","2017,",Tennessee,Inactive


In [6]:
#split the column
df[['Draft Round', 'Draft Year']] = df['Drafted Details'].str.split(', ', 1, expand=True)
df.drop('Drafted Details',axis='columns', inplace=True)
df

,Player Name,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year
0,Dak Prescott,"$450,000",5 Years,Cowboys,QB,27-24d,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016
1,Aaron Rodgers,"$230,000",16 Years,Packers,QB,37-20d,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005
2,Matthew Stafford,"$3,100,000",12 Years,Rams,QB,33-15d,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009
3,Joey Bosa,"$450,000",5 Years,Chargers,DE,25-11d,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016
4,Jimmy Garoppolo,"$420,000",7 Years,49ers,QB,29-20d,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014
...,...,...,...,...,...,...,...,...,...,...,...
4267,Colby Gossett,-,2 Years,Browns,G,25-29d,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018
4268,Jeremiah Valoaga,"$300,883",3 Years,Raiders,DE,26-7d,"2017, Lions",Nevada-Las Vegas,Inactive,Undrafted,Undrafted
4269,Hunter Sharp,"$23,750",1 Years,Bengals,WR,27-27d,"2016, Eagles",Utah State,Inactive,Undrafted,Undrafted
4270,LaTroy Lewis,"$86,400",1 Years,Titans,ILB,27-13d,"2017,",Tennessee,Inactive,Undrafted,Undrafted


In [7]:
df.dropna(subset=['Age','Agent','College','Team','Position'], inplace = True)
df.reset_index(drop = True, inplace = True)
df.isnull().sum()

Player Name           0
Salary                0
Experience Years      0
Team                  0
Position              0
Age                   0
College               0
Agent                 0
Player Status       357
Draft Round           0
Draft Year            0
dtype: int64

In [8]:
#for age in years
for i in range(len(df['Age'])):
    df['Age'][i] = df['Age'][i].split("-")[0]
df

,Player Name,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year
0,Dak Prescott,"$450,000",5 Years,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016
1,Aaron Rodgers,"$230,000",16 Years,Packers,QB,37,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005
2,Matthew Stafford,"$3,100,000",12 Years,Rams,QB,33,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009
3,Joey Bosa,"$450,000",5 Years,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016
4,Jimmy Garoppolo,"$420,000",7 Years,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014
...,...,...,...,...,...,...,...,...,...,...,...
4113,Colby Gossett,-,2 Years,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018
4114,Jeremiah Valoaga,"$300,883",3 Years,Raiders,DE,26,"2017, Lions",Nevada-Las Vegas,Inactive,Undrafted,Undrafted
4115,Hunter Sharp,"$23,750",1 Years,Bengals,WR,27,"2016, Eagles",Utah State,Inactive,Undrafted,Undrafted
4116,LaTroy Lewis,"$86,400",1 Years,Titans,ILB,27,"2017,",Tennessee,Inactive,Undrafted,Undrafted


In [9]:
#for number only
for i in range(len(df['Experience Years'])):
    df['Experience Years'][i] = df['Experience Years'][i].split(" ")[0]
df

,Player Name,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year
0,Dak Prescott,"$450,000",5,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016
1,Aaron Rodgers,"$230,000",16,Packers,QB,37,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005
2,Matthew Stafford,"$3,100,000",12,Rams,QB,33,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009
3,Joey Bosa,"$450,000",5,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016
4,Jimmy Garoppolo,"$420,000",7,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014
...,...,...,...,...,...,...,...,...,...,...,...
4113,Colby Gossett,-,2,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018
4114,Jeremiah Valoaga,"$300,883",3,Raiders,DE,26,"2017, Lions",Nevada-Las Vegas,Inactive,Undrafted,Undrafted
4115,Hunter Sharp,"$23,750",1,Bengals,WR,27,"2016, Eagles",Utah State,Inactive,Undrafted,Undrafted
4116,LaTroy Lewis,"$86,400",1,Titans,ILB,27,"2017,",Tennessee,Inactive,Undrafted,Undrafted


In [10]:
#fill empty cells in "Experience Years" column
df["Experience Years"] = pd.to_numeric(df["Experience Years"])
 
for i in range(len(df["Experience Years"])):
    if df["Experience Years"][i] == "":
        df["Experience Years"][i] = np.nan

df["Experience Years"].fillna(df["Experience Years"].mean(), inplace = True)

In [11]:
#split the column
df[['First Name', 'Last Name']] = df['Player Name'].str.split(' ', 1, expand=True)
df.drop('Player Name',axis='columns', inplace=True)
df

,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year,First Name,Last Name
0,"$450,000",5.0,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016,Dak,Prescott
1,"$230,000",16.0,Packers,QB,37,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005,Aaron,Rodgers
2,"$3,100,000",12.0,Rams,QB,33,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009,Matthew,Stafford
3,"$450,000",5.0,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016,Joey,Bosa
4,"$420,000",7.0,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014,Jimmy,Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...
4113,-,2.0,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018,Colby,Gossett
4114,"$300,883",3.0,Raiders,DE,26,"2017, Lions",Nevada-Las Vegas,Inactive,Undrafted,Undrafted,Jeremiah,Valoaga
4115,"$23,750",1.0,Bengals,WR,27,"2016, Eagles",Utah State,Inactive,Undrafted,Undrafted,Hunter,Sharp
4116,"$86,400",1.0,Titans,ILB,27,"2017,",Tennessee,Inactive,Undrafted,Undrafted,LaTroy,Lewis


In [12]:
#fill empty cells in 'Player Status' column 
df.fillna({'Player Status':'Inactive'},inplace = True)
df

,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year,First Name,Last Name
0,"$450,000",5.0,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016,Dak,Prescott
1,"$230,000",16.0,Packers,QB,37,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005,Aaron,Rodgers
2,"$3,100,000",12.0,Rams,QB,33,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009,Matthew,Stafford
3,"$450,000",5.0,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016,Joey,Bosa
4,"$420,000",7.0,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014,Jimmy,Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...
4113,-,2.0,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018,Colby,Gossett
4114,"$300,883",3.0,Raiders,DE,26,"2017, Lions",Nevada-Las Vegas,Inactive,Undrafted,Undrafted,Jeremiah,Valoaga
4115,"$23,750",1.0,Bengals,WR,27,"2016, Eagles",Utah State,Inactive,Undrafted,Undrafted,Hunter,Sharp
4116,"$86,400",1.0,Titans,ILB,27,"2017,",Tennessee,Inactive,Undrafted,Undrafted,LaTroy,Lewis


In [13]:
#replcing char for easier handling
for i in range(len(df["Salary"])):
    if df["Salary"][i] == "-":
        df["Salary"][i] = ""

In [14]:
#for numbers only
for i in range(len(df["Salary"])):
    if "$" in df["Salary"][i]:
        df["Salary"][i] = df["Salary"][i].split("$")[1]

In [15]:
#removing all "," 
for j in range(3):
    for i in range(len(df["Salary"])):
        if "," in df["Salary"][i]:
            df["Salary"][i] = df["Salary"][i].replace(",","",1)

In [16]:
#fill empty cells in column "Salary"
df["Salary"] = pd.to_numeric(df["Salary"])
 
for i in range(len(df["Salary"])):
    if df["Salary"][i] == "":
        df["Salary"][i] = np.nan

df["Salary"].fillna(df["Salary"].mean(), inplace = True)

In [17]:
df["Age"] = pd.to_numeric(df["Age"])

In [18]:
df

,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year,First Name,Last Name
0,4.500000e+05,5.0,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016,Dak,Prescott
1,2.300000e+05,16.0,Packers,QB,37,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005,Aaron,Rodgers
2,3.100000e+06,12.0,Rams,QB,33,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009,Matthew,Stafford
3,4.500000e+05,5.0,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016,Joey,Bosa
4,4.200000e+05,7.0,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014,Jimmy,Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...
4113,3.838200e+05,2.0,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018,Colby,Gossett
4114,3.008830e+05,3.0,Raiders,DE,26,"2017, Lions",Nevada-Las Vegas,Inactive,Undrafted,Undrafted,Jeremiah,Valoaga
4115,2.375000e+04,1.0,Bengals,WR,27,"2016, Eagles",Utah State,Inactive,Undrafted,Undrafted,Hunter,Sharp
4116,8.640000e+04,1.0,Titans,ILB,27,"2017,",Tennessee,Inactive,Undrafted,Undrafted,LaTroy,Lewis


In [19]:
#fill empty cells
for i in range(len(df["Agent"])):
    if df["Agent"][i] == " ":
        df["Agent"][i] = "Unknown"
df

,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year,First Name,Last Name
0,4.500000e+05,5.0,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016,Dak,Prescott
1,2.300000e+05,16.0,Packers,QB,37,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005,Aaron,Rodgers
2,3.100000e+06,12.0,Rams,QB,33,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009,Matthew,Stafford
3,4.500000e+05,5.0,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016,Joey,Bosa
4,4.200000e+05,7.0,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014,Jimmy,Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...
4113,3.838200e+05,2.0,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018,Colby,Gossett
4114,3.008830e+05,3.0,Raiders,DE,26,"2017, Lions",Nevada-Las Vegas,Inactive,Undrafted,Undrafted,Jeremiah,Valoaga
4115,2.375000e+04,1.0,Bengals,WR,27,"2016, Eagles",Utah State,Inactive,Undrafted,Undrafted,Hunter,Sharp
4116,8.640000e+04,1.0,Titans,ILB,27,"2017,",Tennessee,Inactive,Undrafted,Undrafted,LaTroy,Lewis


In [20]:
#for college name only
for i in range(len(df["College"])):
    if "," in df["College"][i]:
        df["College"][i] = df["College"][i].split(",")[1]

#fill empty cells
for i in range(len(df["College"])):
    if df["College"][i] == " ":
        df["College"][i] = "Unknown"
        
df

,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year,First Name,Last Name
0,4.500000e+05,5.0,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),Active,Round 4 (#135 overall),2016,Dak,Prescott
1,2.300000e+05,16.0,Packers,QB,37,California,David Dunn (Athletes First),Active,Round 1 (#24 overall),2005,Aaron,Rodgers
2,3.100000e+06,12.0,Rams,QB,33,Georgia,Tom Condon (CAA Sports),Active,Round 1 (#1 overall),2009,Matthew,Stafford
3,4.500000e+05,5.0,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),Active,Round 1 (#3 overall),2016,Joey,Bosa
4,4.200000e+05,7.0,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),Active,Round 2 (#62 overall),2014,Jimmy,Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...
4113,3.838200e+05,2.0,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",Active,Round 6 (#213 overall),2018,Colby,Gossett
4114,3.008830e+05,3.0,Raiders,DE,26,Lions,Nevada-Las Vegas,Inactive,Undrafted,Undrafted,Jeremiah,Valoaga
4115,2.375000e+04,1.0,Bengals,WR,27,Eagles,Utah State,Inactive,Undrafted,Undrafted,Hunter,Sharp
4116,8.640000e+04,1.0,Titans,ILB,27,Unknown,Tennessee,Inactive,Undrafted,Undrafted,LaTroy,Lewis


In [21]:
#replacing data to boolean values
replace_status = {'Active':1,'Inactive':0}
df.replace(replace_status, inplace = True)
df

,Salary,Experience Years,Team,Position,Age,College,Agent,Player Status,Draft Round,Draft Year,First Name,Last Name
0,4.500000e+05,5.0,Cowboys,QB,27,Mississippi State,Todd France (CAA Sports),1,Round 4 (#135 overall),2016,Dak,Prescott
1,2.300000e+05,16.0,Packers,QB,37,California,David Dunn (Athletes First),1,Round 1 (#24 overall),2005,Aaron,Rodgers
2,3.100000e+06,12.0,Rams,QB,33,Georgia,Tom Condon (CAA Sports),1,Round 1 (#1 overall),2009,Matthew,Stafford
3,4.500000e+05,5.0,Chargers,DE,25,Ohio State,Brian Ayrault (CAA Sports),1,Round 1 (#3 overall),2016,Joey,Bosa
4,4.200000e+05,7.0,49ers,QB,29,Eastern Illinois,Don Yee (Yee & Dubin Sports),1,Round 2 (#62 overall),2014,Jimmy,Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...
4113,3.838200e+05,2.0,Browns,G,25,Appalachian State,"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",1,Round 6 (#213 overall),2018,Colby,Gossett
4114,3.008830e+05,3.0,Raiders,DE,26,Lions,Nevada-Las Vegas,0,Undrafted,Undrafted,Jeremiah,Valoaga
4115,2.375000e+04,1.0,Bengals,WR,27,Eagles,Utah State,0,Undrafted,Undrafted,Hunter,Sharp
4116,8.640000e+04,1.0,Titans,ILB,27,Unknown,Tennessee,0,Undrafted,Undrafted,LaTroy,Lewis


In [22]:
#remove empty cells
for i in range(len(df["Player Status"])):
    if df["Player Status"][i] != 1 and df["Player Status"][i] != 0:
        df["Player Status"][i] = np.nan

df.dropna(subset=['Player Status'], inplace = True)

In [23]:
df = df[['First Name','Last Name','College','Salary','Team','Position','Age','Experience Years','Draft Year','Draft Round','Agent','Player Status']]
df

,First Name,Last Name,College,Salary,Team,Position,Age,Experience Years,Draft Year,Draft Round,Agent,Player Status
0,Dak,Prescott,Mississippi State,4.500000e+05,Cowboys,QB,27,5.0,2016,Round 4 (#135 overall),Todd France (CAA Sports),1
1,Aaron,Rodgers,California,2.300000e+05,Packers,QB,37,16.0,2005,Round 1 (#24 overall),David Dunn (Athletes First),1
2,Matthew,Stafford,Georgia,3.100000e+06,Rams,QB,33,12.0,2009,Round 1 (#1 overall),Tom Condon (CAA Sports),1
3,Joey,Bosa,Ohio State,4.500000e+05,Chargers,DE,25,5.0,2016,Round 1 (#3 overall),Brian Ayrault (CAA Sports),1
4,Jimmy,Garoppolo,Eastern Illinois,4.200000e+05,49ers,QB,29,7.0,2014,Round 2 (#62 overall),Don Yee (Yee & Dubin Sports),1
...,...,...,...,...,...,...,...,...,...,...,...,...
4113,Colby,Gossett,Appalachian State,3.838200e+05,Browns,G,25,2.0,2018,Round 6 (#213 overall),"Billy Conaty, Edward Johnson, Tony Agnone (Eas...",1
4114,Jeremiah,Valoaga,Lions,3.008830e+05,Raiders,DE,26,3.0,Undrafted,Undrafted,Nevada-Las Vegas,0
4115,Hunter,Sharp,Eagles,2.375000e+04,Bengals,WR,27,1.0,Undrafted,Undrafted,Utah State,0
4116,LaTroy,Lewis,Unknown,8.640000e+04,Titans,ILB,27,1.0,Undrafted,Undrafted,Tennessee,0


In [24]:
df.isnull().sum()

First Name          0
Last Name           0
College             0
Salary              0
Team                0
Position            0
Age                 0
Experience Years    0
Draft Year          0
Draft Round         0
Agent               0
Player Status       0
dtype: int64

In [25]:
df.to_csv("Cleaned_NFLPlayerDetails.csv",index = False)

In [26]:
df_no_outlier = outlier_detection_iqr(df)
df_no_outlier.to_csv("Cleaned_NFLPlayerDetails_NoOutliers.csv",index = False)